In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Student data

In [5]:
staff_data_df = pd.read_sql('select * from staff', conn)
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [6]:
staff_data_df.rename(columns={
    'ID_Staff':'ID_Staff__c',
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'BirthDate__c'
}, inplace=True)

staff_data_df

,ID_Staff__c,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [7]:
staff_data_df['BirthDate__c'] = pd.to_datetime(staff_data_df['BirthDate__c']).dt.date

In [8]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'LastName__c': 'Luongo',
  'FirstName__c': 'Darick',
  'MiddleName__c': 'Nico',
  'BirthDate__c': datetime.date(1995, 7, 5)},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': datetime.date(1992, 4, 22)},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': datetime.date(1993, 5, 21)},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'LastName__c': 'Rodney',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Nicole',
  'BirthDate__c': datetime.date(1994, 6, 7)}]

In [ ]:
for rec in staff_data_records:
    record = {
        'ID_Staff__c': rec['ID_Staff__c'],
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': str(rec['BirthDate__c'])
    }
    
    try:
        sf.Staff__C.create(record)
    except Exception as e:
        print(e)

### Create Staff Lookup Table

In [9]:
staff_lookup_list = []

data = sf.query_all_iter('select ID_Staff__c, Name from Staff__c')

for row in data:
    rec = {
        'Staff_ID__c': row['Name'],
        'ID_Staff__c': row['ID_Staff__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'Staff_ID__c': 'a044x0000016WiU', 'ID_Staff__c': '4'},
 {'Staff_ID__c': 'a044x0000016WiK', 'ID_Staff__c': '2'},
 {'Staff_ID__c': 'a044x0000016WiF', 'ID_Staff__c': '1'},
 {'Staff_ID__c': 'a044x0000016WiP', 'ID_Staff__c': '3'}]

In [10]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,Staff_ID__c,ID_Staff__c
0,a044x0000016WiU,4
1,a044x0000016WiK,2
2,a044x0000016WiF,1
3,a044x0000016WiP,3


In [11]:
staff_lookup_df['ID_Staff__c'] = staff_lookup_df['ID_Staff__c'].astype(str)
staff_lookup_df

,Staff_ID__c,ID_Staff__c
0,a044x0000016WiU,4
1,a044x0000016WiK,2
2,a044x0000016WiF,1
3,a044x0000016WiP,3


### Create Class lookup table

In [12]:
class_lookup_list = []

data = sf.query_all_iter('select ID_Class__c, Name from Class__c')
for row in data:
    rec = {
        "Class_ID__c": row['Name'],
        "ID_Class__c": row['ID_Class__c']
    }
    
    class_lookup_list.append(rec)

class_lookup_list

[{'Class_ID__c': 'a014x000008Tj9e', 'ID_Class__c': '2'},
 {'Class_ID__c': 'a014x000008Tj9Z', 'ID_Class__c': '1'},
 {'Class_ID__c': 'a014x000008Tj9j', 'ID_Class__c': '3'},
 {'Class_ID__c': 'a014x000008Tj9o', 'ID_Class__c': '4'}]

In [13]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df.head()

,Class_ID__c,ID_Class__c
0,a014x000008Tj9e,2
1,a014x000008Tj9Z,1
2,a014x000008Tj9j,3
3,a014x000008Tj9o,4


In [14]:
class_lookup_df['ID_Class__c'] = class_lookup_df['ID_Class__c'].astype(str)
class_lookup_df

,Class_ID__c,ID_Class__c
0,a014x000008Tj9e,2
1,a014x000008Tj9Z,1
2,a014x000008Tj9j,3
3,a014x000008Tj9o,4


### Query the Staffasignment table from MySQL

In [24]:
query = '''
   select * from staffassignment
'''

staffassignment_data_df = pd.read_sql(query, conn)
staffassignment_data_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-09-16,None
1,2,4,1,Teacher Assistant,2020-09-16,None
2,3,3,1,Instructor,2020-09-16,None


In [25]:
staffassignment_data_df.rename(columns={
    'ID_StaffAssignment':'ID_StaffAssignment__c',
    'ID_Staff':'ID_Staff__c',
    'ID_Class':'ID_Class__c',
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)

staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-09-16,None
1,2,4,1,Teacher Assistant,2020-09-16,None
2,3,3,1,Instructor,2020-09-16,None


In [26]:
staffassignment_data_df['StartDate__c'] = pd.to_datetime(staffassignment_data_df['StartDate__c']).dt.date
staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-09-16,None
1,2,4,1,Teacher Assistant,2020-09-16,None
2,3,3,1,Instructor,2020-09-16,None


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [27]:
staff_lookup_df

,Staff_ID__c,ID_Staff__c
0,a044x0000016WiU,4
1,a044x0000016WiK,2
2,a044x0000016WiF,1
3,a044x0000016WiP,3


In [29]:
staffassignment_data_df['ID_Staff__c'] = staffassignment_data_df['ID_Staff__c'].astype(str)

In [30]:
staffassignment_data1 = pd.merge(staffassignment_data_df, staff_lookup_df, how='left')
staffassignment_data1

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Staff_ID__c
0,1,1,1,Teacher Assistant,2020-09-16,None,a044x0000016WiF
1,2,4,1,Teacher Assistant,2020-09-16,None,a044x0000016WiU
2,3,3,1,Instructor,2020-09-16,None,a044x0000016WiP


In [31]:
# convert float to integer
#class_lookup_df['ID_Class__c'] = class_lookup_df['ID_Class__c'].astype(str)
staffassignment_data1['ID_Class__c'] = staffassignment_data1['ID_Class__c'].astype(str)

In [32]:
staffassignment_data2 = pd.merge(staffassignment_data1, class_lookup_df, how='left')
staffassignment_data2

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Staff_ID__c,Class_ID__c
0,1,1,1,Teacher Assistant,2020-09-16,None,a044x0000016WiF,a014x000008Tj9Z
1,2,4,1,Teacher Assistant,2020-09-16,None,a044x0000016WiU,a014x000008Tj9Z
2,3,3,1,Instructor,2020-09-16,None,a044x0000016WiP,a014x000008Tj9Z


In [ ]:
#staffassignment_data2['StartDate__c'] = pd.to_datetime(staffassignment_data2['StartDate__c']).dt.date

In [35]:
staffassignment_data2['EndDate__c'] = pd.to_datetime(staffassignment_data2['EndDate__c']).dt.date

In [36]:
staffassignment_data_records = staffassignment_data2.to_dict(orient='records')
staffassignment_data_records

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': '1',
  'ID_Class__c': '1',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': NaT,
  'Staff_ID__c': 'a044x0000016WiF',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_StaffAssignment__c': 2,
  'ID_Staff__c': '4',
  'ID_Class__c': '1',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': NaT,
  'Staff_ID__c': 'a044x0000016WiU',
  'Class_ID__c': 'a014x000008Tj9Z'},
 {'ID_StaffAssignment__c': 3,
  'ID_Staff__c': '3',
  'ID_Class__c': '1',
  'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': NaT,
  'Staff_ID__c': 'a044x0000016WiP',
  'Class_ID__c': 'a014x000008Tj9Z'}]

## Insert `Staffassignment` Records into Salesforce

In [38]:
for rec in staffassignment_data_records:
    record = {
        'ID_StaffAssignment__c': rec['ID_StaffAssignment__c'],
        'ID_Staff__c': rec['ID_Staff__c'],
        'ID_Class__c': rec['ID_Class__c'],
        'Role__c': rec['Role__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'EndDate__c': str(rec['EndDate__c']),
        'Staff_ID__c': rec['Staff_ID__c'],
        'Class_ID__c': rec['Class_ID__c']
    }
    
    try:
        sf.Staffassignment__C.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [ ]:
staff_records = sf.query("SELECT Id FROM Staff__c")
recs_to_delete = [{'Id': r['Id']} for r in staff_records['records']]
recs_to_delete

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [ ]:
for rec in recs_to_delete:
    try:
        sf.staff__c.delete(rec['Id'])
    except Exception as e:
        print(e)